# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, linked_variables: Dict[str, str] = {}) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negat

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, linked_variables={})

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'linked_variables': {}}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables": {"a": [0.0, 1000.0], "b": [-1.0, 1.0]}, "constraints": {"e": ["LESS_THAN", 2.0], "f": ["GREATER_THAN", 0.0]}, "objectives": {"c": "MAXIMIZE", "d": "MINIMIZE"}, "constants": {"g": 1234}, "linked_variables": {}}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,188.970056,-0.792101,1234,188.177954,189.762157,376.355909,379.524314
6,3.480680,-0.939362,1234,2.541318,4.420042,5.082635,8.840084
7,287.940322,0.334659,1234,288.274981,287.605663,576.549961,575.211325
8,868.124521,0.541603,1234,868.666124,867.582918,1737.332248,1735.165836
9,855.707474,-0.703046,1234,855.004428,856.410519,1710.008856,1712.821038
10,799.588032,0.350950,1234,799.938983,799.237082,1599.877965,1598.474164
11,297.915733,-0.739705,1234,297.176028,298.655437,594.352056,597.310875
12,199.045937,0.230079,1234,199.276016,198.815858,398.552033,397.631716
13,900.562662,0.874427,1234,901.437089,899.688236,1802.874179,1799.376471
14,277.106852,0.665110,1234,277.771962,276.441742,555.543925,552.883484


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-188.177954,189.762157
6,-2.541318,4.420042
7,-288.274981,287.605663
8,-868.666124,867.582918
9,-855.004428,856.410519
10,-799.938983,799.237082
11,-297.176028,298.655437
12,-199.276016,198.815858
13,-901.437089,899.688236
14,-277.771962,276.441742


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-188.177954,189.762157
6,-2.541318,4.420042
7,-288.274981,287.605663
8,-868.666124,867.582918
9,-855.004428,856.410519
10,-799.938983,799.237082
11,-297.176028,298.655437
12,-199.276016,198.815858
13,-901.437089,899.688236
14,-277.771962,276.441742


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-188.17795435,  189.76215713],
       [  -2.54131759,    4.42004193],
       [-288.27498067,  287.60566254],
       [-868.66612425,  867.58291785],
       [-855.00442789,  856.41051923],
       [-799.93898255,  799.23708194],
       [-297.17602783,  298.65543739],
       [-199.27601641,  198.81585794],
       [-901.43708925,  899.68823574],
       [-277.77196247,  276.44174185]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,374.355909,-379.524314
6,3.082635,-8.840084
7,574.549961,-575.211325
8,1735.332248,-1735.165836
9,1708.008856,-1712.821038
10,1597.877965,-1598.474164
11,592.352056,-597.310875
12,396.552033,-397.631716
13,1800.874179,-1799.376471
14,553.543925,-552.883484


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
